In [3]:
import pandas as pd
import numpy as np

In [4]:
ApparelData = pd.read_csv('apparel-trainval.csv',sep=',',index_col = False)

In [13]:
#UniqueLabels=np.array([0,1,2,3,4,5,6,7,8,9])
UniqueLabels=np.array([5,6])
DataPerClass = []
index = 0
for i in UniqueLabels:
    tempdf = ApparelData.loc[ApparelData['label'] == i]
    DataPerClass.append(tempdf.sample(frac = 1))
    print(len(DataPerClass[index]))#+str(" for ")+str(i))
    index = index + 1
#train = pd.concat(DataPerClass, ignore_index= True)
#train = train.sample(frac = 1)
#val = ApparelData.loc[~ApparelData.index.isin(train.index)]
#val = val.sample(frac = 1)

6000
6000


In [19]:
train5 = DataPerClass[0].sample(frac=0.8)
train6 = DataPerClass[1].sample(frac=0.8)
val5 = DataPerClass[0].loc[~DataPerClass[0].index.isin(train5.index)]
val6 = DataPerClass[1].loc[~DataPerClass[1].index.isin(train6.index)]
allTrain = [train5,train6]
allVal = [val5,val6]
train = pd.concat(allTrain, ignore_index= True)
val = pd.concat(allVal, ignore_index= True)
train = train.sample(frac=1)
val = val.sample(frac=1)
len(train)
#len(val)

9600

In [20]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
4013,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,172,134,0,0
2610,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
770,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7168,6,0,0,0,120,185,164,182,182,168,...,76,84,69,69,84,0,0,0,0,0
2572,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
Xmatrix = ApparelData.values
LabelVector = Xmatrix[:,0]

In [23]:
Xmatrix.shape

(9600, 784)

In [21]:
Xmatrix = train.values
LabelVector = Xmatrix[:,0]
Xmatrix = Xmatrix[:,1:]
meanVector = np.mean(Xmatrix,axis = 0)
CenteredXmatrix = Xmatrix - meanVector
SdVector = np.std(CenteredXmatrix.astype(float),axis = 0)
CenteredXmatrix /= SdVector
covOfCenteredXmatrix = np.array([(CenteredXmatrix[0,:].astype(float))]).T@ np.array([(CenteredXmatrix[0,:].astype(float))])
for i in range(1,len(CenteredXmatrix)):
    if(i%10==0):
        print(i)
    covOfCenteredXmatrix +=   np.array([(CenteredXmatrix[i,:].astype(float))]).T@ np.array([(CenteredXmatrix[i,:].astype(float))])
covOfCenteredXmatrix /= len(CenteredXmatrix)
eigenValues, eigenVectors = np.linalg.eig(covOfCenteredXmatrix)
idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[:,idx]
neededEigenVectors = []
for idx, val in enumerate(eigenValues):
    if (sum(eigenValues[0:idx])/sum(eigenValues))>0.9:
        neededEigenVectors = eigenVectors[:,0:idx]
        break
reconstructedXmatrix = CenteredXmatrix.astype(float)@neededEigenVectors

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [22]:
reconstructedXmatrix.shape

(9600, 151)

In [33]:
len(LabelVector)

9600

In [ ]:
#np.cov(CenteredXmatrix).shape

In [ ]:
#newData = np.insert(reconstructedXmatrix.astype(str), 2, values=LabelVector, axis=1) 

In [5]:
reconstructedXmatrixDict = {}
reconstructedXmatrixDict[0] = reconstructedXmatrix

In [5]:
import pickle

In [8]:
pickle_out = open("ReconXmat.pickle","wb")
pickle.dump(reconstructedXmatrixDict, pickle_out)
pickle_out.close()

In [6]:
pickle_in = open("ReconXmat.pickle","rb")
reconstructedXmatrixDict = pickle.load(pickle_in)

In [26]:
LabelVector.reshape(-1,1)

array([[5],
       [5],
       [5],
       ...,
       [5],
       [6],
       [6]], dtype=int64)

In [24]:
#reconstructedXmatrix = np.insert(reconstructedXmatrix.astype(str), 0, values=LabelVector, axis=1) 

In [25]:
reconstructedXmatrix.shape

(9600, 152)

In [28]:
#newData = pd.DataFrame(data=reconstructedXmatrix, columns=list(ApparelData.columns)[:133]) 

In [18]:
newData.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel123,pixel124,pixel125,pixel126,pixel127,pixel128,pixel129,pixel130,pixel131,pixel132
0,3,3.963666271141668,12.818833487711506,-3.1049325882102927,0.7206015758759704,-0.7788570122819052,-2.076487727783692,2.5496456882339062,-1.2314700737192037,-1.0054774147158818,...,0.7674857567165779,0.7910644345195357,-0.25877068231201183,1.4871123042013483,0.371779035942733,0.7099468559455187,1.7981830808782577,-0.6145413352099546,0.3590085871179435,-0.10291305730615782
1,0,-9.514157176516614,9.986658260724273,6.256852306320593,-8.213848909771478,-4.985408054931778,-1.461835408825586,-5.1566358374271495,-1.560220771340061,1.36903018553951,...,-0.14848674714816057,-0.3250402602623607,-0.3065047525402313,0.43361931078783944,-0.2870512120839133,-0.17701519729885254,0.14049283695161546,-0.47446158617195355,-0.011783538138638874,0.765362950956593
2,9,-10.8483756559792,-11.366745886050378,-7.143213477509138,-1.483078342880324,2.430131872878139,2.1244626135367515,-5.389361829042253,-4.037629620892771,0.6256697063933858,...,0.35950466571008943,0.8661800893934514,0.006693106020129402,-0.8673824941468109,-0.5973181251556414,-0.17079414799583592,-1.1869799799099343,0.22389284075358823,0.3805179282684352,-0.20342104410230405
3,0,-19.300076988661665,8.567805499702448,9.209350752096793,-5.430799208622463,-0.7136217960866948,-2.8662998529647687,1.0041333504282197,-4.292446219935175,-0.913686262468919,...,-0.1492325625224344,-0.16400662108712996,0.03436136158902588,-0.001195024118773702,-0.36311194150406517,-0.1888292362714418,0.007722666601598177,0.12258400499615602,-0.009348266032546466,-0.4315417768715597
4,3,5.545525859505426,14.084653579381479,-10.914439130091429,-0.23544781302734386,2.1901473188816207,-3.123085247031823,3.8270612322203164,0.8994869215727597,0.22723525939288092,...,0.7664645232107368,-0.18087500550241628,-1.0941185775271716,-0.6443675335478485,0.08844343855081394,0.3796010322699558,-0.1830755019781196,-0.06949094626386249,-0.13587859673495847,1.1261237875255696


In [34]:
from sklearn.svm import SVC
clf = SVC(C = 10, kernel = 'linear')
clf.fit(reconstructedXmatrix[:,1:], LabelVector) 

print('w = ',clf.coef_)
print('b = ',clf.intercept_)
print('Indices of support vectors = ', clf.support_)
print('Support vectors = ', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))

w =  [[ 3.82339472e-01 -2.35536941e-01  1.17079483e-01 -2.40361042e-02
  -2.81559311e-01 -1.10329736e-01  2.23848298e-01  1.98474297e-01
  -1.70805378e-02 -1.35997538e-01  5.65871287e-03 -3.99184212e-02
  -4.34680950e-02 -1.54464338e-01  5.10627283e-02  1.31130517e-01
  -5.12009103e-02  1.01725662e-01 -3.53106636e-02  1.64392809e-01
   8.01809224e-02  1.03551473e-01  1.06790447e-01 -3.40343936e-02
   1.51500624e-01 -7.82223487e-02 -1.28946810e-01  3.64424986e-02
  -8.38349929e-02  1.40973652e-01  5.99930279e-02  8.80387307e-02
   3.84379116e-02 -3.52824679e-02  1.36352806e-02 -7.61278982e-02
   8.20742462e-02  3.80898016e-02  1.19978883e-01  5.94566117e-02
   3.18199133e-02  1.12272659e-01 -1.12174979e-01  5.82345448e-02
   1.86453306e-01  8.22251965e-02 -5.55144041e-02 -2.98437298e-03
  -4.56592018e-02  1.09236913e-02  5.66755980e-02 -6.00728215e-02
   3.10307617e-02  7.42620705e-03  8.13339791e-02 -1.56732889e-01
  -8.09420700e-02 -1.51566170e-02  8.35110378e-02 -3.91659340e-02
   1.

In [41]:
PredSVM = []
for i in range(0,len(train)):
    PredSVM.append(clf.predict([reconstructedXmatrix[i,1:]]))

In [45]:
PredSVM1 = np.array(PredSVM).ravel()

In [46]:
conMatrix = pd.crosstab(PredSVM1,LabelVector)

In [47]:
conMatrix

col_0,5,6
row_0,,
5,4800,0
6,0,4800


In [39]:
#clf.predict([reconstructedXmatrix[20,1:]])

In [ ]:
https://xavierbourretsicotte.github.io/SVM_implementation.html

In [ ]:
https://towardsdatascience.com/support-vector-machine-mnist-digit-classification-with-python-including-my-hand-written-digits-83d6eca7004a

In [ ]:
https://towardsdatascience.com/a-simple-example-of-pipeline-in-machine-learning-with-scikit-learn-e726ffbb6976 

In [ ]:
https://towardsdatascience.com/understanding-support-vector-machine-part-2-kernel-trick-mercers-theorem-e1e6848c6c4d